# Proyecto Final
## Jason Rodriguez
## Percy Herrera

El set de datos es sobre informacion de campana de recoleccion de dineros de cuentas en mora, es la informacion sobre clientes contactados, con una oferta concreta para cerrar su cuenta en mora por un monto inferior al actual.

In [ ]:
import pandas as pd

df1 = pd.read_csv('proyectoF1T.csv')
df2 = pd.read_csv('proyectoF2F.csv')
df1.head()